# Colab demo — MLOps SentimentNotebook pronto per Google Colab: clona il repo, installa le dipendenze, addestra il modello, valuta/promuove in MLflow e prova un'inferenza.

## 1) Clona il repository
Imposta l'URL del tuo fork pubblico (variabile `REPO_URL`, default già settato) e clona in `/content`. Se hai già il repo montato, salta questa cella.


In [ ]:
import os, pathlib
REPO_URL = os.environ.get('REPO_URL', 'https://github.com/<your-account>/MachineInnovatorsInc_ProAI.git')
repo_dir = pathlib.Path('/content/MachineInnovatorsInc_ProAI')

if not repo_dir.exists():
    !git clone $REPO_URL
%cd $repo_dir



## 2) Installa le dipendenzeUsa i requisiti del progetto (transformers, scikit-learn, mlflow, ecc.).

In [ ]:
import sys, pathlib, os
project_root = pathlib.Path.cwd()
if not (project_root / 'src').exists():
    alt = pathlib.Path('/content/MachineInnovatorsInc_ProAI')
    if alt.exists():
        project_root = alt
        os.chdir(project_root)
    else:
        raise RuntimeError('Repository non trovato: esegui la cella 1 per clonare e posizionarti nella repo.')
else:
    os.chdir(project_root)

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

!pip install -q -r requirements.txt

os.environ.setdefault('MLFLOW_TRACKING_URI', 'file://' + str(project_root/'mlruns'))
os.environ.setdefault('REGISTERED_MODEL_NAME', 'Sentiment')

print('Working directory:', project_root)


## 3) Smoke test di inferenza (Hugging Face)Usa la pipeline pre-addestrata `cardiffnlp/twitter-roberta-base-sentiment-latest` senza passare da MLflow.

In [ ]:
from src.serving.load_model import predict_fn
label, score = predict_fn('I love this course!')
print({'label': label, 'score': score})


## 4) Addestra e registra una nuova versione in MLflowQuesto step logga una versione del modello nel Model Registry locale (file-based).

In [ ]:
!python -m src.models.train_roberta --experiment sentiment_colab


## 5) Recupera l'ultima versione registrataLeggiamo dal Model Registry l'ultima versione (stage `None`) per valutarla.

In [ ]:
from mlflow.tracking import MlflowClient
model_name = os.environ.get('REGISTERED_MODEL_NAME', 'Sentiment')
client = MlflowClient(tracking_uri=os.environ['MLFLOW_TRACKING_URI'])
versions = client.get_latest_versions(model_name, stages=['None'])
if not versions:
    versions = client.search_model_versions(f"name='{model_name}'")
latest = sorted(versions, key=lambda v: int(v.version))[-1]
new_model_uri = f"models:/{model_name}/{latest.version}"
print('new_model_uri =', new_model_uri)


## 6) Valuta e promuovi rispetto all'holdoutUsiamo `data/holdout.csv` come set di valutazione. Se il nuovo modello è >= del Production (o se Production non esiste) viene promosso.

In [ ]:
!python -m src.models.evaluate --new_model_uri $new_model_uri --eval_csv data/holdout.csv --min_improvement 0.0


## 7) Leggi il modello in ProductionOtteniamo la URI del modello con stage `Production` dal Registry e impostiamo `MODEL_URI` per l'inferenza.

In [ ]:
from src.utils.mlflow_utils import get_production_model_uri
prod_uri = get_production_model_uri(model_name)
print('Production URI:', prod_uri)
os.environ['MODEL_URI'] = prod_uri or ''


## 8) Inferenza usando il modello in ProductionCon `MODEL_URI` puntato al registry, `predict_fn` userà MLflow; in fallback userà Hugging Face.

In [ ]:
label, score = predict_fn('This is a terrible experience')
print({'label': label, 'score': score})


## 9) Note utili- Il registry è file-based (`mlruns` nella root): scarica la cartella se vuoi conservarlo.- Per simulare data drift e triggerare il retrain via Airflow/Grafana, vedi `docs/data_drift_simulation.md`.- Il flusso di training/valutazione/promozione è descritto in `docs/training_and_promotion.md`.